In [1]:
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import time
import re
import random

# Initialisation du générateur de nombres aléatoires
random.seed(123)

tokenizer = nltk.RegexpTokenizer(r'''\w'|\w+|[^\w\s]''')

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Dense,Dropout,Embedding,LSTM
# from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [7]:
import nltk

# Télécharge les données nécessaires pour le tokenizer (segmenteur) de phrases et de mots
nltk.download('punkt')

# Télécharge la liste de mots vides (stopwords) pour différentes langues
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\samir\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\samir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
train_data = pd.read_csv('../cleaned_data/train_cleaned.csv',index_col=0)
dev_data   = pd.read_csv('../cleaned_data/dev_cleaned.csv',index_col=0)
test_data  =  pd.read_csv('../cleaned_data/test_cleaned.csv',index_col=0)

In [11]:
train_data

,movie,review_id,name,user_id,note,commentaire
0,229864,review_58145147,AM11,Z20051105131449940367618,7,trouve face tient côté cette phrase tagline fi...
1,229864,review_59303833,this is my movies,Z20051220181957240298347,6,soi rien contre genre récit hyper codifié déjà...
2,229864,review_61916672,Shephard69,Z20121226180301760912584,8,troisième réalisation après tron héritage obli...
3,229864,review_1000190786,Tayop,Z20121022182542230468657,5,joseph kosinski offre nouvelles images ébourif...
4,229864,review_59207755,ned123,Z20061211223247607682175,6,film traite drame humain plus haute intensité ...
...,...,...,...,...,...,...
665957,245551,review_55820729,Wally L,Z20121109004006553333277,7,super realisation quelques bon acteurs cette v...
665958,245551,review_1001840194,r@ch,Z20100726110927837577369,5,film visuellement très beau mis musique rap am...
665959,245551,review_56242470,Kaiser D.,Z20170721223303710182536,6,parti histoire parait banale très bon film tou...
665960,245551,review_58174196,Florent A.,Z20161204004443590736237,5,mitigé film coté plutot accroché peu mal aise ...


In [12]:
type(train_data['commentaire'][0])

str

In [47]:
# train_data_t = pd.read_csv('../data/train.csv')
# # dev_data   = pd.read_csv('../data/dev.csv')  .replace('.',',')

# train_data_t['note'] = train_data_t['note'].str.replace(',', '.').astype(float)
# train_data_t['note'] = train_data_t['note']*2

# train_data_t

In [18]:
train_data['commentaire'] = train_data['commentaire'].astype(str)
dev_data['commentaire'] = dev_data['commentaire'].astype(str)
test_data['commentaire'] = test_data['commentaire'].astype(str)

In [19]:
train_comments = train_data['commentaire'].values.tolist()
dev_comments   = dev_data['commentaire'].values.tolist()
test_comments  = test_data['commentaire'].values.tolist()


In [21]:
tokenized_sentences = test_comments+dev_comments+train_comments

In [22]:
tokenized_sentences

['jours film raté sortie vois maintenant sans doute recul meilleurs films depardon revient hôpital psychiatrique trente ans après urgences filmer cette fois gens présentés devant juge savoir celui autorise hospitalisation sous contrainte comme habitude dispositif très sobre sobre soigné depardon photographe voit chacun plans ici documentaire joue reportage investigation bouge caméra tous sens ici très calme permet spectateur familiariser personnes comprendre viennent quel problème humaniser quelque sorte juste vus comme fous spectateurs forcément film touchant puisqu montre vrais gens gens souffrent gens envie sortir gens font tout parfois avoir air plus normal possible pouvoir sortir suffit jamais film verra jamais juge refuser autorisation médecins poursuivre hospitalisation sous contrainte malades rêvent liberté finir systématiquement réalité condition mentale revient réellement tragique voit voit problème propos incohérents sent nervosité comprend peuvent sortir temps voit aussi hô

# Corpus Tokenization

In [23]:
from nltk.tokenize import word_tokenize

tokens = []
length = len(tokenized_sentences)
for i in range(length):
    tokens.append(word_tokenize(tokenized_sentences[i]))

In [24]:
tokens

[['jours',
  'film',
  'raté',
  'sortie',
  'vois',
  'maintenant',
  'sans',
  'doute',
  'recul',
  'meilleurs',
  'films',
  'depardon',
  'revient',
  'hôpital',
  'psychiatrique',
  'trente',
  'ans',
  'après',
  'urgences',
  'filmer',
  'cette',
  'fois',
  'gens',
  'présentés',
  'devant',
  'juge',
  'savoir',
  'celui',
  'autorise',
  'hospitalisation',
  'sous',
  'contrainte',
  'comme',
  'habitude',
  'dispositif',
  'très',
  'sobre',
  'sobre',
  'soigné',
  'depardon',
  'photographe',
  'voit',
  'chacun',
  'plans',
  'ici',
  'documentaire',
  'joue',
  'reportage',
  'investigation',
  'bouge',
  'caméra',
  'tous',
  'sens',
  'ici',
  'très',
  'calme',
  'permet',
  'spectateur',
  'familiariser',
  'personnes',
  'comprendre',
  'viennent',
  'quel',
  'problème',
  'humaniser',
  'quelque',
  'sorte',
  'juste',
  'vus',
  'comme',
  'fous',
  'spectateurs',
  'forcément',
  'film',
  'touchant',
  'puisqu',
  'montre',
  'vrais',
  'gens',
  'gens',
  'so

# W2V

In [25]:
from gensim.models import Word2Vec

Word2Vec_model = Word2Vec(tokens, vector_size=100, min_count=1,epochs = 10)

Word2Vec_model.save("Word2Vec.model")

In [26]:
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec

Word2Vec_model = Word2Vec.load("Word2Vec.model")

In [27]:
train_list = [' '.join(comment.split()[:100]) for comment in train_comments if comment]
dev_list = [' '.join(comment.split()[:100]) for comment in dev_comments if comment]

In [18]:
# train_list

In [28]:
train_data['nouvelle commentaire'] = train_list
dev_data['nouvelle commentaire']   = dev_list

In [29]:
train_data['nouvelle commentaire']

0         trouve face tient côté cette phrase tagline fi...
1         soi rien contre genre récit hyper codifié déjà...
2         troisième réalisation après tron héritage obli...
3         joseph kosinski offre nouvelles images ébourif...
4         film traite drame humain plus haute intensité ...
                                ...                        
665957    super realisation quelques bon acteurs cette v...
665958    film visuellement très beau mis musique rap am...
665959    parti histoire parait banale très bon film tou...
665960    mitigé film coté plutot accroché peu mal aise ...
665961    bon film netflix fois scénario simple acteurs ...
Name: nouvelle commentaire, Length: 665962, dtype: object

In [30]:
X_train = train_data['nouvelle commentaire']
X_dev   = dev_data['nouvelle commentaire']

tokenizer = Tokenizer(num_words=len(list(X_train.unique())))
tokenizer.fit_on_texts(list(X_train))

X_train = tokenizer.texts_to_sequences(X_train) 
X_dev = tokenizer.texts_to_sequences(X_dev)

X_train = pad_sequences(X_train, maxlen=100)
X_dev = pad_sequences(X_dev, maxlen=100)

In [31]:
X_train

array([[ 124,  268,  232, ...,  976,    2, 1350],
       [ 815,   26,  144, ..., 9741, 1952, 5437],
       [ 625,   90,   45, ..., 1203,   66,   28],
       ...,
       [   0,    0,    0, ...,   35,  299, 2314],
       [   0,    0,    0, ...,   39,  306,  440],
       [   0,    0,    0, ...,  294,    6,    1]])

In [32]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

204672

In [33]:
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    try:
        vector = Word2Vec_model.wv[word]
    except:
        continue
        print(i)
    embedding_matrix[i] = vector

In [34]:
embedding_matrix

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 2.13824511e+00, -1.25981355e+00, -1.26042998e+00, ...,
         3.40708584e-01, -9.31206286e-01, -1.72567821e+00],
       [-1.08294198e-02, -7.53580868e-01, -1.27702308e+00, ...,
         2.75413775e+00, -3.58427614e-01, -4.20683533e-01],
       ...,
       [ 6.61246777e-02,  1.94796264e-01,  7.81791210e-02, ...,
         3.01404130e-02,  7.28372834e-04,  4.06194367e-02],
       [-2.88064359e-03, -1.44484760e-02, -2.00113803e-02, ...,
        -4.55475710e-02, -2.08389387e-03,  4.57564779e-02],
       [ 2.13663001e-02, -3.80801670e-02, -2.56709531e-02, ...,
         8.06524325e-03, -7.04700351e-02,  1.25309944e-01]])

In [35]:
y_train = np.array(train_data['note'])
y_dev = np.array(dev_data['note'])

In [36]:
y_dev

array([9, 6, 7, ..., 7, 0, 0], dtype=int64)

In [37]:
y_train = y_train.astype(int)
y_dev = y_dev.astype(int)
y_dev

array([9, 6, 7, ..., 7, 0, 0])

In [38]:
num_classes = 10

In [39]:
model = Sequential()
model.add(tf.keras.layers.Embedding(vocab_size,
                            100,
                            weights=[embedding_matrix],
                            input_length=100,
                            trainable=True))
model.add(tf.keras.layers.Flatten(input_shape=(102, 100)))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 100, 100)          20467200  
                                                                 
 flatten (Flatten)           (None, 10000)             0         
                                                                 
 dense (Dense)               (None, 128)               1280128   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 512)               66048     
                                                                 
 dropout_1 (Dropout)         (None, 512)               0         
                                                                 
 dense_2 (Dense)             (None, 256)               1

In [40]:
import time
from tensorflow.keras.callbacks import ModelCheckpoint

st = time.time()

#create callback
filepath = 'MLP_Word2Vec_Best_One.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath, 
                             monitor='val_accuracy',
                             verbose=1, 
                             save_best_only=True,
                             mode='max')
callbacks = [checkpoint]


history=model.fit(X_train, y_train, validation_data=(X_dev, y_dev),epochs=20, batch_size=128, verbose=1,callbacks=callbacks)

elapsed_time = time.time() - st
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

Epoch 1/20
5203/5203 [==============================] - ETA: 0s - loss: 1.8046 - accuracy: 0.2924
Epoch 1: val_accuracy improved from -inf to 0.34308, saving model to MLP_Word2Vec_Best_One.hdf5


C:\Users\samir\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5203/5203 [==============================] - 1838s 353ms/step - loss: 1.8046 - accuracy: 0.2924 - val_loss: 1.7001 - val_accuracy: 0.3431
Epoch 2/20
5203/5203 [==============================] - ETA: 0s - loss: 1.6822 - accuracy: 0.3327
Epoch 2: val_accuracy improved from 0.34308 to 0.35320, saving model to MLP_Word2Vec_Best_One.hdf5
5203/5203 [==============================] - 2002s 385ms/step - loss: 1.6822 - accuracy: 0.3327 - val_loss: 1.6549 - val_accuracy: 0.3532
Epoch 3/20
5203/5203 [==============================] - ETA: 0s - loss: 1.6437 - accuracy: 0.3450
Epoch 3: val_accuracy did not improve from 0.35320
5203/5203 [==============================] - 1952s 375ms/step - loss: 1.6437 - accuracy: 0.3450 - val_loss: 1.6390 - val_accuracy: 0.3529
Epoch 4/20
5203/5203 [==============================] - ETA: 0s - loss: 1.6187 - accuracy: 0.3530
Epoch 4: val_accuracy improved from 0.35320 to 0.35350, saving model to MLP_Word2Vec_Best_One.hdf5
5203/5203 [==============================] 

In [44]:
from tensorflow.keras.models import load_model
# model = load_model('MLP_Best_One.hdf5')



modelpath = "C:/trained_Models/MLP_Word2Vec_Best_One.hdf5"

# Load the model with compile=False
MLP_Word2Vec_Best_One = load_model(modelpath)


In [76]:
# model

In [45]:
# test_data['commentaire'] = test_data['commentaire'].astype(str)
X_test = test_data['commentaire']


In [46]:
X_test = tokenizer.texts_to_sequences(X_test)
X_test = pad_sequences(X_test, maxlen=100)

In [47]:
predictions = MLP_Word2Vec_Best_One.predict(X_test)

2683/2683 [==============================] - 29s 10ms/step


In [48]:
predictions

array([[0.16562288, 0.08826606, 0.07030079, ..., 0.14915971, 0.10383219,
        0.1262907 ],
       [0.0041893 , 0.00534721, 0.01032681, ..., 0.30388525, 0.11303446,
        0.08052954],
       [0.00384712, 0.00463286, 0.00862968, ..., 0.32519296, 0.13779722,
        0.10837455],
       ...,
       [0.00065055, 0.00252125, 0.00850828, ..., 0.14717822, 0.02467234,
        0.00980387],
       [0.00170526, 0.001311  , 0.00205288, ..., 0.32317707, 0.22647183,
        0.27474368],
       [0.00124462, 0.00095337, 0.00155375, ..., 0.3230471 , 0.23250142,
        0.2837469 ]], dtype=float32)

In [50]:
argmax_predictions = np.argmax(predictions,axis =1)

In [51]:
argmax_predictions

array([0, 7, 7, ..., 5, 7, 7], dtype=int64)

In [53]:
notes = [] #génération du fichier pour la plateforme d'évaluation
with open("MLP_Word2Vec_ID_Prediction.txt", "w") as f:
    for i in range(len(test_data['review_id'])):
        notes.append((argmax_predictions[i]+1)/2)
        f.write(str(test_data['review_id'].iloc[i])+' '+str(notes[i]).replace('.',',')+'\n')